In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import urllib.request
import boto3
from collections import Counter
from konlpy.tag import Kkma

In [2]:
def replace_empty_with_nan(text):
    if text.strip() == '':
        return np.nan
    else:
        return text

### 이모지랑 한글만 가져오도록 하기

In [3]:
total = pd.read_csv('./data/naver_result_0305.csv', encoding_errors='ignore', low_memory=False)
total_data = total.copy()

for i, body in total_data['body'].items():
    if isinstance(body, str):
        total_data.at[i, 'body'] = re.sub(r'[\U00010000-\U0010ffff]', '', body)
    else:
        total_data.at[i, 'body'] = np.nan
total_data = total_data.dropna(subset=['body'])
total_data.drop_duplicates(subset=['body'], inplace=True)
total_data['body'] = total_data['body'].apply(lambda x: re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', str(x)))
total_data['body'] = total_data['body'].fillna('')
total_data['body'] = total_data['body'].replace('', np.nan)
total_data['body'] = total_data['body'].str.replace('\n', '')

# 문자열에 strip() 메소드 적용
total_data['body'] = total_data['body'].str.strip()
total_data['body'].replace('', np.nan, inplace=True) 
total_data = total_data.dropna(subset=['body'])
total_data['body'] = total_data['body'].astype(str)

In [4]:
# 형태소 분석기 및 불용어 정의
kkma = Kkma()
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [5]:
# 새로운 열을 초기화하고 각각의 품사에 따라 토큰을 저장
total_data['nouns'] = None
total_data['verbs'] = None
total_data['adjectives'] = None
total_data['exclamations'] = None

In [6]:
from tqdm import tqdm

In [7]:
# 형태소 분석 함수 정의
def tokenize_text(text):
    nouns = [word for word, pos in kkma.pos(text) if pos.startswith('N')]
    verbs = [word for word, pos in kkma.pos(text) if pos.startswith('V')]
    adjectives = [word for word, pos in kkma.pos(text) if pos.startswith('VA')]
    exclamations = [word for word, pos in kkma.pos(text) if pos.startswith('E')]
    return nouns, verbs, adjectives, exclamations

In [9]:
total_data['rating'].replace('nan', np.nan, inplace=True)
total_data.dropna(subset=['rating'], inplace=True)

In [11]:
total_data['label'] = None

In [12]:
# rating 열을 숫자로 변환합니다.
total_data['rating'] = pd.to_numeric(total_data['rating'], errors='coerce')

# 숫자로 변환된 rating 열에서 4 이상인 행을 찾아서 label을 1로 설정합니다.
total_data.loc[total_data['rating'] >= 4, 'label'] = 1

# 숫자로 변환된 rating 열에서 2 이하인 행을 찾아서 label을 0으로 설정합니다.
total_data.loc[total_data['rating'] <= 2, 'label'] = 0

# label이 할당되지 않은 행을 제거합니다.
total_data = total_data.dropna(subset=['label'])

In [13]:
total_data = total_data[['label','body']].copy()

In [14]:
# DataFrame의 기본 인덱스를 사용하여 인덱스를 다시 설정합니다.
total_data.reset_index(drop=True, inplace=True)

In [15]:
# 열 이름 변경
total_data = total_data.rename(columns={'label': 'Rating', 'body': 'Review'})

In [16]:
total_data

,Rating,Review
0,0,손님 놓치기 싫어서 지하층으로 손님을 받을거면 동일한 서비스를 제공하세요 그동안 ...
1,1,김치찌개랑 계란말이 쨩
2,1,맛있어요
3,1,바빠요
4,1,삼굡살이 너무맛나게먹엇쳐욤
...,...,...
10481,1,주차 시간 무료
10482,1,케익이 맛있는 아티제
10483,1,주차가돼요
10484,1,깔끔하고 저녁엔 회사 사람들이 빠져나가서 조용하고 좋네요


In [17]:
total_data.Rating.value_counts()

Rating
1    9968
0     518
Name: count, dtype: int64

In [18]:
total_data.Review.str.len().describe()

count    10486.000000
mean        40.985600
std         50.059683
min          1.000000
25%         12.000000
50%         24.000000
75%         50.000000
max        397.000000
Name: Review, dtype: float64

In [21]:
# 형태소 분석 결과를 데이터프레임에 저장
total_data['nouns'], total_data['verbs'], total_data['adjectives'], total_data['exclamations'] = zip(*[tokenize_text(text) for text in tqdm(total_data['Review'])])

print(total_data)

100%|██████████| 10486/10486 [22:22<00:00,  7.81it/s]

      Rating                                             Review  \
0          0  손님 놓치기 싫어서 지하층으로 손님을 받을거면 동일한 서비스를 제공하세요  그동안 ...   
1          1                                       김치찌개랑 계란말이 쨩   
2          1                                               맛있어요   
3          1                                                바빠요   
4          1                                     삼굡살이 너무맛나게먹엇쳐욤   
...      ...                                                ...   
10481      1                                           주차 시간 무료   
10482      1                                        케익이 맛있는 아티제   
10483      1                                              주차가돼요   
10484      1                    깔끔하고 저녁엔 회사 사람들이 빠져나가서 조용하고 좋네요   
10485      1                                       삼성타워 크고 좋음 ㅎ   

                                                   nouns  \
0      [손님, 지하층, 손님, 거, 동일, 서비스, 제공, 앞, 번, 사장님, 남자, 분...   
1                                          [김치찌개, 계란, 말]   
2              

In [22]:
total_data

,Rating,Review,nouns,verbs,adjectives,exclamations
0,0,손님 놓치기 싫어서 지하층으로 손님을 받을거면 동일한 서비스를 제공하세요 그동안 ...,"[손님, 지하층, 손님, 거, 동일, 서비스, 제공, 앞, 번, 사장님, 남자, 분...","[놓치, 싫, 받, 이, 가, 갈, 이, 하, 달라, 하, 주, 없, 눈치보, 시키...","[싫, 없]","[기, 어서, 을, 면, ㄴ, 세요, 었, 지만, ㅂ니다, ㄴ지, 시, 던데, 고,..."
1,1,김치찌개랑 계란말이 쨩,"[김치찌개, 계란, 말]",[],[],[]
2,1,맛있어요,[],[맛있],[맛있],[어요]
3,1,바빠요,[],[바쁘],[바쁘],[아요]
4,1,삼굡살이 너무맛나게먹엇쳐욤,[],"[맛나, 치]",[맛나],"[게, 어요]"
...,...,...,...,...,...,...
10481,1,주차 시간 무료,"[주차, 시간, 무료]",[],[],[]
10482,1,케익이 맛있는 아티제,"[케익, 티]","[맛있, 아]",[맛있],"[는, 아]"
10483,1,주차가돼요,[주차],[되],[],[어요]
10484,1,깔끔하고 저녁엔 회사 사람들이 빠져나가서 조용하고 좋네요,"[저녁, 회사, 사람]","[빠져나가, 좋]",[좋],"[고, 서, 고, 네요]"


In [23]:
total_data.to_csv('./data/total_tokenized_V0.csv', index=False, encoding='utf-8-sig')